#  1. 영어 Word2Vec 만들기

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

# 훈련데이터 이해하기

In [ ]:
# XML데이터이므로 전처리 필요
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

# 훈련데이터 전처리하기

In [ ]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

# Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)
# 만약 TypeError: __init__() got an unexpected keyword argument 'size' 라는 에러 발생 시에는
# size 대신 vector_size로 바꿔서 적어주세요.

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

# Word2Vec 모델 저장하고 로드하기

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

#  2. 한국어 Word2Vec 만들기(토탈훈련데이터)

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 6.0 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 58.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset

/content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset


In [ ]:
import json
with open('KoNLPy(Mecab)_stopwords(18)_Total(movie&shopping&game&target)_tokened_vocab_v0.1.json', encoding='UTF8') as json_file:
    tokenized_data = json.load(json_file)

In [ ]:
print(len(tokenized_data))
tokenized_data

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v') # 모델 저장

In [ ]:
print(model.wv.most_similar("최민식"))

In [ ]:
print(model.wv.most_similar("히어로"))

In [ ]:
print(model.wv.most_similar("꺼져"))

In [ ]:
print(model.wv.most_similar("짱"))

#  3. 한국어 Word2Vec 만들기(위키피디아)

In [12]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset/Word2Vec

/content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset/Word2Vec


1) 위키피디아 한국어 덤프 파일 다운로드

https://dumps.wikimedia.org/kowiki/latest/

kowiki-latest-pages-articles.xml.bz2 

2) 위키피디아 익스트랙터 다운로드

In [11]:
!git clone "https://github.com/attardi/wikiextractor.git"  

fatal: destination path 'wikiextractor' already exists and is not an empty directory.


3) 위키피디아 한국어 덤프 파일 변환

In [ ]:
python WikiExtractor.py kowiki-latest-pages-articles.xml.bz2  

# 4) 훈련 데이터 만들기

copy AA디렉토리의 경로\wiki* wikiAA.txt